## Porting GoldenRetriever to TF2

### 1. USE-QA in TF2

Official code sample for TF2  
https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/2

In [1]:
# !pip install tensorflow-addons
# !pip install tensorflow_text
# !pip install --upgrade tensorflow-hub

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
import numpy as np
import tensorflow_text

questions = ["What is your age?"]
responses = ["I am 20 years old.", "good morning"]
response_contexts = ["I will be 21 next year.", "great day."]

module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/2')

question_embeddings = module.signatures['question_encoder'](
            tf.constant(questions))
response_embeddings = module.signatures['response_encoder'](
        input=tf.constant(responses),
        context=tf.constant(response_contexts))

np.inner(question_embeddings['outputs'], response_embeddings['outputs'])

array([[0.40884   , 0.08877401]], dtype=float32)

### 2. Inference in TF2 model
Noticeably, it runs faster than previously.

In [4]:
import sys
sys.path.append('..')

from src.model import GoldenRetriever
print("timing init")
%time gr = GoldenRetriever(**{'learning_rate':0.001, 'beta_1':0.9, 'beta_2':0.999})
print("")


# encode 1 question
print("timing ques encoding")
%time encoded_ques = gr.predict('How old are you?', type='query')
print("")

# encode multiple questions
encoded_ques = gr.predict(['How old are you?', 'What time is it?'], 
                          type='query')

# one response w context
print("timing response encoding")
%time encoded_res = gr.predict("I am 20 years old.", context="I will be 21 next year.", type='response')
print("")

# multiple responses w/0 context
encoded_res = gr.predict(["I am 20 years old.", "I love apple cider"], type='response')



timing init
model initiated!
CPU times: user 14.2 s, sys: 867 ms, total: 15 s
Wall time: 14.9 s

timing ques encoding
CPU times: user 2.09 s, sys: 38.7 ms, total: 2.13 s
Wall time: 2.08 s

timing response encoding
CPU times: user 2.34 s, sys: 15.2 ms, total: 2.36 s
Wall time: 2.31 s



### 3. Loading knowledge bases

In [5]:
# load knowledge bases
gr.load_kb(path_to_kb='../data/aiap.txt', is_faq=True, kb_name='aiap')
gr.load_kb(path_to_kb='../data/resale_tnc.txt', kb_name='resale_tnc')
gr.load_kb(path_to_kb='../data/fund_guide_tnc_full.txt', kb_name='nrf')
gr.load_csv_kb(path_to_kb='../data/pdpa.csv', cutoff=196, kb_name='pdpa')

# make query
gr.make_query('What kind of candidates are you looking for?', top_k=2, kb_name='aiap')


knowledge base lock and loaded!
knowledge base lock and loaded!
knowledge base lock and loaded!
knowledge base (csv) lock and loaded!


(['We are looking for candidates who possess a keen interest in the area of machine learning and data science. We believe that candidates can come from any area of specialisation, and our requirements are as follow:\ni)   Singaporean with a polytechnic diploma or university degree,\nii) Proficient in Python or R and iii) Is able to implement Machine Learning Algorithms or have a background in Mathematics / Statistics / Computer Science. \nBeyond that, demonstrated statistical fundamentals and programming ability will be helpful for the technical tests, but a keen learning attitude will be the most important to carry you through the programme. \n',
  'Candidates can expect to be equipped in some or all of the following skills: data modelling/tuning, data engineering, data product-related software engineering, cloud applications. It ranges between individuals, but candidates can be adequately prepared in fields of data science, engineering and consultancy\n'],
 array([[0.3085595 ],
     

### 4. Testing finetuning

In [6]:
"""
1. Cosine loss
"""
print("BEFORE FINETUNING")
print(gr.var_finetune)
print("")

questions = ["What is your age?"]
responses = ["I am 20 years old.", "good morning"]
response_contexts = ["I will be 21 next year.", "great day."]
%time gr.finetune(questions, responses, response_contexts, label=[1,0], loss='cosine')

print("AFTER FINETUNING")
print(gr.var_finetune)


"""
2. Contrastive loss
"""
print("BEFORE FINETUNING")
print(gr.var_finetune)
print("")

questions = ["What is your age?"]
responses = ["I am 20 years old.", "good morning"]
response_contexts = ["I will be 21 next year.", "great day."]
%time gr.finetune(questions, responses, response_contexts, label=[1,0], loss='contrastive')

print("AFTER FINETUNING")
print(gr.var_finetune)


"""
3. Triplet loss
"""
print("BEFORE FINETUNING")
print(gr.var_finetune)
print("")

questions = ["What is your age?"]
responses = ["The top section of the spine is damaged."]
response_contexts = ["Call the nurse."]
%time gr.finetune(questions, responses, response_contexts, neg_answer = ["I will be 21 years old."], neg_answer_context = ["Time is running out for the elderly and the young."], loss='triplet')

print("AFTER FINETUNING")
print(gr.var_finetune)


BEFORE FINETUNING
[<tf.Variable 'QA/Final/Response_tuning/ResidualHidden_1/AdjustDepth/projection/kernel:0' shape=(1024, 512) dtype=float32, numpy=
array([[ 1.7117115e+00, -8.8389181e-03, -8.8152960e-02, ...,
         3.4359641e-02, -5.6950748e-02,  3.1913319e-03],
       [-1.7560545e-02,  1.5315282e+00,  2.8173655e-02, ...,
         2.7225253e-03,  1.5698759e-02, -8.1266584e-03],
       [-4.7530245e-02,  4.8905790e-02,  1.6913337e+00, ...,
        -3.7007492e-02, -3.5331409e-02, -9.1013429e-04],
       ...,
       [-4.3463507e-03,  1.1113880e-02,  1.0710205e-02, ...,
         5.0418127e-02,  3.3716073e-03, -2.2927163e-02],
       [ 2.6251100e-02, -5.4154057e-02, -1.5461433e-02, ...,
        -8.8774767e-03, -2.9500483e-02,  4.1329965e-02],
       [ 1.5746625e-02, -2.4634020e-02, -2.4179602e-02, ...,
         1.6309937e-02,  1.0962348e-02, -3.4185573e-02]], dtype=float32)>]

CPU times: user 29.3 s, sys: 316 ms, total: 29.6 s
Wall time: 29.4 s
AFTER FINETUNING
[<tf.Variable 'QA/Final/Res

The last few values in the array has changed, indicating that there is successful tuning.

### 5. Testing Exporting and Restoring model

In [2]:
import sys
sys.path.append('..')

from src.model import GoldenRetriever
print("timing init")
%time gr = GoldenRetriever(**{'learning_rate':0.001, 'beta_1':0.9, 'beta_2':0.999})
print("")


gr.export("fine_tune")
gr.restore("fine_tune")

print("AFTER EXPORTING AND RESTORING")
print(gr.var_finetune)

timing init
model initiated!
CPU times: user 14 s, sys: 914 ms, total: 14.9 s
Wall time: 14.9 s

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: fine_tune/assets


INFO:tensorflow:Assets written to: fine_tune/assets


AFTER EXPORTING AND RESTORING
[<tf.Variable 'QA/Final/Response_tuning/ResidualHidden_1/AdjustDepth/projection/kernel:0' shape=(1024, 512) dtype=float32, numpy=
array([[ 1.7117115e+00, -8.8389181e-03, -8.8152960e-02, ...,
         3.4359641e-02, -5.6950748e-02,  3.1913319e-03],
       [-1.7560545e-02,  1.5315282e+00,  2.8173655e-02, ...,
         2.7225253e-03,  1.5698759e-02, -8.1266584e-03],
       [-4.7530245e-02,  4.8905790e-02,  1.6913337e+00, ...,
        -3.7007492e-02, -3.5331409e-02, -9.1013429e-04],
       ...,
       [-4.3463507e-03,  1.1113880e-02,  1.0710205e-02, ...,
         5.0418127e-02,  3.3716073e-03, -2.2927163e-02],
       [ 2.6251100e-02, -5.4154057e-02, -1.5461433e-02, ...,
        -8.8774767e-03, -2.9500483e-02,  4.1329965e-02],
       [ 1.5746625e-02, -2.4634020e-02, -2.4179602e-02, ...,
         1.6309937e-02,  1.0962348e-02, -3.4185573e-02]], dtype=float32)>]


The loaded weights are the same post-finetuning weights, suggesting that the exporting and restoring worked fine.

### 6. Testing predicting after restoring weights

In [2]:
# encode 1 question
print("timing ques encoding")
%time encoded_ques = gr.predict('How old are you?', type='query')
print("")

# encode multiple questions
encoded_ques = gr.predict(['How old are you?', 'What time is it?'], 
                          type='query')

# one response w context
print("timing response encoding")
%time encoded_res = gr.predict("I am 20 years old.", context="I will be 21 next year.", type='response')
print("")

# multiple responses w/0 context
encoded_res = gr.predict(["I am 20 years old.", "I love apple cider"], type='response')



timing ques encoding
CPU times: user 2.39 s, sys: 69.8 ms, total: 2.46 s
Wall time: 2.54 s

timing response encoding
CPU times: user 2.51 s, sys: 7.38 ms, total: 2.52 s
Wall time: 2.48 s



### 7. Testing finetuning after restoring

In [1]:
import sys
sys.path.append('..')

from src.model import GoldenRetriever
print("timing init")
%time gr = GoldenRetriever(**{'learning_rate':0.001, 'beta_1':0.9, 'beta_2':0.999})
print("")


gr.export("fine_tune")
gr.restore("fine_tune")

"""
1. Cosine loss
"""
print("BEFORE FINETUNING")
print(gr.var_finetune)
print("")

questions = ["What is your age?"]
responses = ["I am 20 years old.", "good morning"]
response_contexts = ["I will be 21 next year.", "great day."]
%time gr.finetune(questions, responses, response_contexts, label=[1,0], loss='cosine')

print("AFTER FINETUNING")
print(gr.var_finetune)


"""
2. Contrastive loss
"""
print("BEFORE FINETUNING")
print(gr.var_finetune)
print("")

questions = ["What is your age?"]
responses = ["I am 20 years old.", "good morning"]
response_contexts = ["I will be 21 next year.", "great day."]
%time gr.finetune(questions, responses, response_contexts, label=[1,0], loss='contrastive')

print("AFTER FINETUNING")
print(gr.var_finetune)


"""
3. Triplet loss
"""
print("BEFORE FINETUNING")
print(gr.var_finetune)
print("")

questions = ["What is your age?"]
responses = ["The top section of the spine is damaged."]
response_contexts = ["Call the nurse."]
%time gr.finetune(questions, responses, response_contexts, neg_answer = ["I will be 21 years old."], neg_answer_context = ["Time is running out for the elderly and the young."], loss='triplet')

print("AFTER FINETUNING")
print(gr.var_finetune)


timing init
model initiated!
CPU times: user 14.2 s, sys: 1.03 s, total: 15.2 s
Wall time: 15.2 s

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: fine_tune/assets


INFO:tensorflow:Assets written to: fine_tune/assets


model initiated!
BEFORE FINETUNING
[<tf.Variable 'QA/Final/Response_tuning/ResidualHidden_1/AdjustDepth/projection/kernel:0' shape=(1024, 512) dtype=float32, numpy=
array([[ 1.7117115e+00, -8.8389181e-03, -8.8152960e-02, ...,
         3.4359641e-02, -5.6950748e-02,  3.1913319e-03],
       [-1.7560545e-02,  1.5315282e+00,  2.8173655e-02, ...,
         2.7225253e-03,  1.5698759e-02, -8.1266584e-03],
       [-4.7530245e-02,  4.8905790e-02,  1.6913337e+00, ...,
        -3.7007492e-02, -3.5331409e-02, -9.1013429e-04],
       ...,
       [-4.3463507e-03,  1.1113880e-02,  1.0710205e-02, ...,
         5.0418127e-02,  3.3716073e-03, -2.2927163e-02],
       [ 2.6251100e-02, -5.4154057e-02, -1.5461433e-02, ...,
        -8.8774767e-03, -2.9500483e-02,  4.1329965e-02],
       [ 1.5746625e-02, -2.4634020e-02, -2.4179602e-02, ...,
         1.6309937e-02,  1.0962348e-02, -3.4185573e-02]], dtype=float32)>]

CPU times: user 36.2 s, sys: 301 ms, total: 36.5 s
Wall time: 36.2 s
AFTER FINETUNING
[<tf.Varia